<br>

# <center> Sudoku Solving using Computer Vision

<br>

## <center> Part 01 : Digits Classification

<br>

---

<br>


<br>

## Colab Configuration

### Mount Google Drive

In [1]:
'''
    This is required if the code runs in Google Colab.
    - this code will mount Google Drive for Colab.
    - the code needs to run only once.
'''

# # uncomment the below code, run and then comment again.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<br>

### Defining Root Directory

In [2]:
# this directory will be used as Root Directory to read/write any file
rootDir = '/content/drive/MyDrive/_ML/Sudoku/'

<br>

## Import Libraries

In [3]:
# importing all the required libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os, random
import cv2

In [4]:
import json

<br>

## Reading DataSet

In [5]:
digitsPath = rootDir + '02. DataSet/Digits/Images'

data = os.listdir(digitsPath)

data_X = []
data_y = []

digit_class_name = []

data_classes = len(data)

print('------------------------------------------')
print(' Reading Digit Dataset . . .')
print('------------------------------------------')

for class_name in range(data_classes):
  digit_class_name.append( str(class_name) )

  print(f'   > Loading Class : {class_name}')
  
  class_path = digitsPath + '/' + str(class_name)
  image_list = os.listdir(class_path)

  for image_name in image_list:
    pic = cv2.imread(class_path + '/' + image_name)
    pic = cv2.resize(pic, (32,32))
    data_X.append(pic)
    data_y.append(class_name)


if len(data_X) == len(data_y):
  print('------------------------------------------')
  print('   Digits loading successfull.')
  print(f'  Total datapoints : {len(data_X)}')
  print('------------------------------------------')

data_X = np.array(data_X)
data_y = np.array(data_y)

------------------------------------------
 Reading Digit Dataset . . .
------------------------------------------
   > Loading Class : 0
   > Loading Class : 1
   > Loading Class : 2
   > Loading Class : 3
   > Loading Class : 4
   > Loading Class : 5
   > Loading Class : 6
   > Loading Class : 7
   > Loading Class : 8
   > Loading Class : 9
------------------------------------------
   Digits loading successfull.
  Total datapoints : 10160
------------------------------------------


<br>

## Spliting DataSet

In [6]:
# importing library
from sklearn.model_selection import train_test_split

# spliting the data into train and test data
# 0.05% data will be used for test and 99.95% for train
train_X, test_X, train_y, test_y = train_test_split(
                                        data_X,
                                        data_y,
                                        test_size = 0.05
                                    )

# spliting further the train data into train and validation data
# 20% data will be used for validation and 80% for train
train_X, valid_X, train_y, valid_y = train_test_split(
                                        train_X,
                                        train_y,
                                        test_size = 0.2
                                    )


# displaying the shape of the splitted data
print(f"  Training Set Shape = {train_X.shape}")
print(f"Validation Set Shape = {valid_X.shape}")
print(f"      Test Set Shape = {test_X.shape}")

  Training Set Shape = (7721, 32, 32, 3)
Validation Set Shape = (1931, 32, 32, 3)
      Test Set Shape = (508, 32, 32, 3)


<br>

## Image Preprocessing

In [7]:
# importing the image processing library
from keras.preprocessing.image import ImageDataGenerator

# Preprocessing the images for NeuralNet
def preprocess(img):
    '''
        This function converts an image to gray scale,
        equalizes the histogram, and normalizes the image.

        Parameter
        ---------
        img
            An image file

        Return
        ------
        ret
            Processed Image
    '''

    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #making image grayscale
    img = cv2.equalizeHist(img) #Histogram equalization to enhance contrast
    img = img/255 #normalizing
    return img

# preprocessing the image data
train_X = np.array( list(map(preprocess, train_X)) )
test_X  = np.array( list(map(preprocess,  test_X)) )
valid_X = np.array( list(map(preprocess, valid_X)) )

# reshaping the images
train_X = train_X.reshape(train_X.shape[0], train_X.shape[1], train_X.shape[2], 1)
test_X  = test_X.reshape( test_X.shape[0],  test_X.shape[1],  test_X.shape[2],  1)
valid_X = valid_X.reshape(valid_X.shape[0], valid_X.shape[1], valid_X.shape[2], 1)

# augmentation
datagen = ImageDataGenerator(
                width_shift_range=0.1, 
                height_shift_range=0.1, 
                zoom_range=0.2, 
                shear_range=0.1, 
                rotation_range=10
            )
datagen.fit(train_X)

In [8]:
from keras.utils.np_utils import to_categorical

# One hot encoding of the labels
train_y = to_categorical(train_y, data_classes)
test_y = to_categorical(test_y, data_classes)
valid_y = to_categorical(valid_y, data_classes)

<br>

## a. Model Design

In [9]:
# importing the required libraries
from keras.models import Sequential
from keras.layers import Activation, Dropout, Dense, Flatten, BatchNormalization, Conv2D, MaxPooling2D
from keras.optimizers import RMSprop

In [10]:
# Creating a Convolutional Neural Network (CNN)

model = Sequential()

model.add((Conv2D(60,(5,5),input_shape=(32, 32, 1) ,padding = 'Same' ,activation='relu')))
model.add((Conv2D(60, (5,5),padding="same",activation='relu')))
model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Dropout(0.25))

model.add((Conv2D(30, (3,3),padding="same", activation='relu')))
model.add((Conv2D(30, (3,3), padding="same", activation='relu')))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(500,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))


In [11]:
# displaying the summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 60)        1560      
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 60)        90060     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 60)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 30)        16230     
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 30)        8130      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 30)         0         
 2D)                                                    

In [12]:
# defining the optimizer
optimizer = RMSprop(
                learning_rate=0.001, 
                rho=0.9, 
                epsilon = 1e-08, 
                decay=0.0
              )

#Compiling the model
model.compile(
    optimizer = optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
  )

<br>

## b. Model Train

In [13]:
# imprting early stopping callbacks from keras
from keras.callbacks import EarlyStopping

# defining early stopping
early_stop = EarlyStopping( 
    monitor = 'val_loss', 
    mode = 'min', 
    min_delta = 0.001, # minimium amount of change to count as an improvement
    verbose = 1, 
    patience = 20,
    restore_best_weights = True, 
  )

In [ ]:
#Fit the model

history = model.fit( 
              datagen.flow(
                  train_X, 
                  train_y, 
                  batch_size=32
              ),
              epochs = 30, 
              validation_data = (valid_X, valid_y),
              verbose = 2,
              steps_per_epoch= 200,
              callbacks = [early_stop]
          )

Epoch 1/30
200/200 - 153s - loss: 0.8709 - accuracy: 0.7013 - val_loss: 0.1016 - val_accuracy: 0.9658 - 153s/epoch - 767ms/step
Epoch 2/30
200/200 - 151s - loss: 0.2915 - accuracy: 0.9081 - val_loss: 0.0577 - val_accuracy: 0.9839 - 151s/epoch - 753ms/step
Epoch 3/30
200/200 - 166s - loss: 0.2000 - accuracy: 0.9395 - val_loss: 0.0522 - val_accuracy: 0.9839 - 166s/epoch - 829ms/step
Epoch 4/30
200/200 - 168s - loss: 0.1570 - accuracy: 0.9522 - val_loss: 0.0234 - val_accuracy: 0.9912 - 168s/epoch - 839ms/step
Epoch 5/30
200/200 - 153s - loss: 0.1369 - accuracy: 0.9567 - val_loss: 0.0317 - val_accuracy: 0.9891 - 153s/epoch - 764ms/step
Epoch 6/30
200/200 - 151s - loss: 0.1204 - accuracy: 0.9644 - val_loss: 0.0274 - val_accuracy: 0.9922 - 151s/epoch - 755ms/step
Epoch 7/30


In [ ]:
# Testing the model on the test set

score = model.evaluate(test_X, test_y, verbose=0)
print('Test Score = ',score[0])
print('Test Accuracy =', score[1])

<br>

## c. Model Evaluaiton

<br>

### a. Visualizing Loss Analysis

In [ ]:
# accessing history
arch = history.history
# model_loss = pd.DataFrame( model.history.history )
model_loss = pd.DataFrame( history.history )

# plotting the history
sns.set_theme(style="darkgrid")
model_loss[['loss', 'val_loss']].plot(figsize=(12,8))

<br>

### b. Classification Report and Confusion Matrix

<br>

#### Preparing Data

In [ ]:
# copying the original X and y test data
X_test_new = test_X
y_test_new = test_y

In [ ]:
def fill_missing_class() :
  '''
    This function will fill any Intent-Class missing in 'y_test'.
    The sklearn Train_Test_Split generates Train and Test data randomly. So any classes can be missed in the test data.
  '''

  # accessing the global numpy array to write
  global X_test_new
  global y_test_new

  # finding the classes in train and test data
  yTrainArgmax = set( train_y.argmax(axis=1) )
  yTestArgmax  = set( test_y.argmax(axis=1) )

  # finding the intent classes missing in 'y_test' by comparing with 'y_train'
  missing_classes = yTrainArgmax.difference(yTestArgmax)
  
  # convering to List
  missing_classes = list(missing_classes)

  # iterating the missing classes 
  for mc in missing_classes:

    # finding the missing classes that exist in 'y_train'
    indices_mc = np.where(train_y.argmax(axis=1) == mc)[0]  # indices of any missing class

    # if any missing class has more then 5 entity then take first 5 otherwise take whatever entity the class has
    if len(indices_mc) > 4 :
      indices_mc = indices_mc[:5]

    # iteraring the indices, and appending intents from 'X_train' and 'y_train' using the indices
    for i in indices_mc:
      X_test_new = np.append(X_test_new, [train_X[i]], axis=0)
      y_test_new = np.append(y_test_new, [train_y[i]], axis=0)


In [ ]:
# filling any intents missing in 'y_test'
fill_missing_class()

<br>

#### Predicting on 'y_test' data

In [ ]:
# prediction on test set
predictions = model.predict(X_test_new)

In [ ]:
# finding the actual classes
y_test_argmax = y_test_new.argmax(axis=1)

# finding the predicted classes
predictions_argmax = predictions.argmax(axis=1)

<br>

#### Classification Report

In [ ]:
# importing the library
from sklearn.metrics import classification_report

# classification report
_CR = classification_report(y_test_argmax, predictions_argmax, target_names=digit_class_name)

# printing the report
print( _CR )

<br>

#### Confusion Matrix

In [ ]:
# importing the library
from sklearn.metrics import confusion_matrix

# calculationg confusion matrix
confusionMat = confusion_matrix(y_test_argmax, predictions_argmax)

In [ ]:
# defining figure size
plt.figure(figsize=(12,15))

# plotting heatmap of confusion matrix
sns.heatmap(confusionMat, square=True, fmt='d', 
            cbar=False, cmap='YlGnBu',  
            xticklabels=digit_class_name,
            yticklabels=digit_class_name,
            annot=True,
            annot_kws={
                "fontsize":12,
                'fontweight': 'bold',
                },
          )

# defining the label names for X-axis and Y-axis
plt.xlabel('Predicted Class', fontsize=14)
plt.ylabel('Actual Class', fontsize=14)

<br>

### c. Prediction on New Data

Creating Data

In [ ]:
# accessing the main data set
_data = data_X
_target = data_y

# defining random position
pos = random.randint(0, len(_data))

# collecting data from main dataset
new_data = _data[pos]

# preprocessing the new data and converting the target to categorical
new_data = preprocess(new_data)
_target = to_categorical(_target, data_classes)


# reshaping the data
new_data = new_data.reshape(-1, 32, 32, 1)

# extracting true result
true_result_list = _target[pos]
true_class  = int( np.argmax(true_result_list) )


Predicting Class for Created Data

In [ ]:
# predicting result for new_data
predicted_class = model.predict(new_data)

# extracting the maximum predicted calss
predicted_class = int( np.argmax(predicted_class, axis=1) )

Extracting Class name

In [ ]:
# extracting predicted class name
predicted_class_name = digit_class_name[predicted_class]
print(f"\n The predicted class is : '{predicted_class_name}'")

# extracting actual class name
true_class_name = digit_class_name[true_class]
print(f"\n The actual class is : '{true_class_name}'")

<br>
<br>

## Save Model

In [ ]:
# defining the name of the model
modelName = 'DigitClassifier.h5'

# creating the path
path = rootDir + '03. Trainned Model/' + modelName

# save model
model.save( path )